In [6]:
import os
import sys
import time
import json
from typing import Dict, List, Iterable

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import ast
import pandas as pd
import numpy as np
import torch
from datetime import datetime, timedelta
# from torch.nn.utils.rnn import pad_sequence
from tensorflow.keras.utils import pad_sequences
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from deepctr_torch.inputs import SparseFeat, DenseFeat, VarLenSparseFeat, get_feature_names
from deepctr_torch.models import *
from scipy import sparse
from scipy.sparse.linalg import svds

# from src import Preprocesser

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DEVICE = torch.device("cpu")

## Preprocess

In [2]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
songs_df = pd.read_csv('../data/songs.csv')
songs_extra_df = pd.read_csv('../data/song_extra_info.csv')
members_df = pd.read_csv('../data/members.csv')

In [3]:
train_test_df = pd.concat([train_df, test_df], ignore_index=True)

In [4]:
# filter songs and members
songs_df = songs_df[songs_df['song_id'].isin(train_test_df['song_id'])]
songs_extra_df = songs_extra_df[songs_extra_df['song_id'].isin(
    train_test_df['song_id'])]
members_df = members_df[members_df['msno'].isin(train_test_df['msno'])]

### Songs

In [5]:
missing_song_ids = train_test_df[~train_test_df['song_id'].
                                 isin(songs_df['song_id'])][['song_id']]

In [6]:
songs_df = pd.concat((songs_df, missing_song_ids), ignore_index=True)

In [7]:
songs_df.columns

Index(['song_id', 'song_length', 'genre_ids', 'artist_name', 'composer',
       'lyricist', 'language'],
      dtype='object')

In [8]:
# encode song ids
song_id_encoder = LabelEncoder()
song_id_encoder.fit(train_test_df['song_id'])
songs_df['song_id'] = song_id_encoder.transform(songs_df['song_id'])
songs_extra_df['song_id'] = song_id_encoder.transform(
    songs_extra_df['song_id'])
train_df['song_id'] = song_id_encoder.transform(train_df['song_id'])
test_df['song_id'] = song_id_encoder.transform(test_df['song_id'])

In [9]:
# standardize song length
songlen_ss = StandardScaler()
songs_df['song_length'].fillna(songs_df['song_length'].median(), inplace=True)
songs_df['song_length'] = songlen_ss.fit_transform(
    songs_df['song_length'].to_numpy().reshape(-1, 1)).reshape(-1)

In [10]:
def map_idx(item_ls: List[str], item2idx: Dict[str, int]):
    for item in item_ls:
        if item not in item2idx:
            item2idx[item] = len(item2idx)

    return list(map(lambda item: item2idx[item], item_ls))

In [11]:
# genre ids
genre2idx = {'[PAD]': 0}
songs_df['genre_ids'].fillna('Unknown', inplace=True)
songs_df['genre_ids'] = songs_df['genre_ids'].astype(str).str.split('|')
songs_df['genre_ids'] = songs_df['genre_ids'].apply(map_idx,
                                                    item2idx=genre2idx)

In [12]:
def name_split(col_name: str):
    return songs_df[col_name].str.split(
        r"and|,|feat\.|featuring|&|\. |\||/|\\|;",
        regex=True).apply(lambda ls: list(
            map(lambda s: s.strip(), filter(lambda s: s.strip(), ls))))

In [13]:
songs_df['artist_name'].fillna("", inplace=True)
songs_df['composer'].fillna("", inplace=True)
songs_df['lyricist'].fillna("", inplace=True)
songs_df['is_featured'] = songs_df['artist_name'].apply(
    lambda x: 1 if 'feat' in str(x) else 0)

In [14]:
songs_df['artist_name'] = name_split('artist_name')
songs_df['composer'] = name_split('composer')
songs_df['lyricist'] = name_split('lyricist')

In [15]:
songs_df['num_artist'] = songs_df['artist_name'].apply(len)
songs_df['num_composer'] = songs_df['composer'].apply(len)
songs_df['num_lyricist'] = songs_df['lyricist'].apply(len)

In [16]:
for col in ['num_artist', 'num_composer', 'num_lyricist']:
    songs_df[col] = StandardScaler().fit_transform(
        songs_df[col].to_numpy().reshape(-1, 1)).reshape(-1)

In [17]:
artist2idx = {'[PAD]': 0}
songs_df['artist_name'] = songs_df['artist_name'].apply(map_idx,
                                                        item2idx=artist2idx)

In [18]:
for col in ['composer', 'lyricist']:
    songs_df[col] = LabelEncoder().fit_transform(
        songs_df[col].apply(lambda x: x[0] if x else np.nan))


In [19]:
songs_df['language'].fillna(-1, inplace=True)
songs_df['language'] = LabelEncoder().fit_transform(songs_df['language'])

In [59]:
item2idx = {'genre_ids': genre2idx, 'artist_name': artist2idx}

In [53]:
with open('../data/item2idx.json', 'w', encoding='utf-8') as f:
    json.dump(item2idx, f)

In [20]:
songs_df.to_csv('../data/prep_songs.csv', encoding='utf-8', index=False)

In [23]:
songs_df

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,is_featured,num_artist,num_composer,num_lyricist
0,95568,-0.007565,[1],[1],48018,17282,1,0,-0.176534,-0.049576,0.580873
1,341401,-0.345950,[2],[2],35491,13935,5,0,-0.176534,1.073085,0.580873
2,104696,-0.114229,[1],[3],51065,25802,5,0,-0.176534,-0.610907,-0.397917
3,271326,0.166725,[1],[4],46606,19715,1,0,-0.176534,-0.049576,0.580873
4,223595,-0.729310,[3],[5],37237,14608,8,0,-0.176534,-0.049576,0.580873
...,...,...,...,...,...,...,...,...,...,...,...
419915,249433,-0.104544,[15],[],51065,25802,0,0,-1.951470,-0.610907,-0.397917
419916,374537,-0.104544,[15],[],51065,25802,0,0,-1.951470,-0.610907,-0.397917
419917,168359,-0.104544,[15],[],51065,25802,0,0,-1.951470,-0.610907,-0.397917
419918,185731,-0.104544,[15],[],51065,25802,0,0,-1.951470,-0.610907,-0.397917


### Members

In [24]:
train_test_df[~train_test_df['msno'].isin(members_df['msno'])]

,msno,song_id,source_system_tab,source_screen_name,source_type,target,id


In [25]:
members_df.columns

Index(['msno', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'expiration_date'],
      dtype='object')

In [26]:
# encode msno
msno_encoder = LabelEncoder()
msno_encoder.fit(members_df['msno'])
members_df['msno'] = msno_encoder.transform(members_df['msno'])
train_df['msno'] = msno_encoder.transform(train_df['msno'])
test_df['msno'] = msno_encoder.transform(test_df['msno'])

In [27]:
# age
members_df['bd'] = members_df['bd'].apply(lambda x: np.nan
                                          if x <= 5 or x >= 75 else x)
members_df['bd'].fillna(members_df['bd'].median(), inplace=True)
members_df['bd'] = MinMaxScaler().fit_transform(
    members_df['bd'].to_numpy().reshape(-1, 1)).reshape(-1)

In [28]:
# encode geograph info
columns = ['city', 'gender', 'registered_via']
for column in columns:
    column_encoder = LabelEncoder()
    members_df[column] = column_encoder.fit_transform(members_df[column])

In [29]:
# preprocess dates
members_df['registration_init_time'] = pd.to_datetime(
    members_df['registration_init_time'].astype(str))
members_df['expiration_date'] = pd.to_datetime(
    members_df['expiration_date'].astype(str))

In [30]:
members_df['registration_init_time'] = members_df[
    'registration_init_time'].apply(lambda x: np.nan
                                    if x < datetime(2000, 1, 1) else x)
members_df['registration_init_time'].fillna(
    members_df['registration_init_time'].min(), inplace=True)

In [31]:
members_df.loc[members_df['expiration_date'] <
               members_df['registration_init_time'],
               'expiration_date'] = np.nan
members_df['expiration_date'].fillna(members_df['expiration_date'].max(),
                                     inplace=True)

In [32]:
dur_mm = MinMaxScaler()
dur_col = (members_df['expiration_date'] - members_df['registration_init_time']
           ).apply(lambda x: 0 if x < timedelta(0) else x.days).fillna(0)
members_df['duration'] = dur_mm.fit_transform(dur_col.to_numpy().reshape(
    -1, 1)).reshape(-1)

In [33]:
members_df['registration_init_time'] = members_df[
    'registration_init_time'].apply(lambda x: x.timestamp())
members_df['registration_init_time'] = MinMaxScaler().fit_transform(
    members_df['registration_init_time'].to_numpy().reshape(-1, 1)).reshape(-1)
members_df['expiration_date'] = members_df['expiration_date'].apply(
    lambda x: x.timestamp())
members_df['expiration_date'] = MinMaxScaler().fit_transform(
    members_df['expiration_date'].to_numpy().reshape(-1, 1)).reshape(-1)

In [34]:
members_df.to_csv('../data/prep_members.csv', encoding='utf-8', index=False)

In [35]:
members_df

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date,duration
0,19016,0,0.298507,2,2,0.572427,0.807903,0.431734
1,17476,0,0.298507,2,2,0.870606,0.792508,0.140804
2,7977,0,0.298507,2,1,0.931597,0.795929,0.088755
3,27021,0,0.298507,2,3,0.885430,0.680636,0.000194
4,29129,0,0.298507,2,1,0.993011,0.790968,0.026801
...,...,...,...,...,...,...,...,...
34398,18725,0,0.298507,2,2,0.744812,0.806192,0.271703
34399,23698,2,0.164179,1,0,0.818297,0.786521,0.181977
34400,10811,0,0.298507,2,2,0.723422,0.805850,0.290930
34401,18326,0,0.298507,2,2,0.894748,0.807903,0.136143


### Train Test

In [40]:
train_test_df.columns

Index(['msno', 'song_id', 'source_system_tab', 'source_screen_name',
       'source_type', 'target', 'id'],
      dtype='object')

In [41]:
# encode source*
columns = ['source_system_tab', 'source_screen_name', 'source_type']
for column in columns:
    column_encoder = LabelEncoder()
    column_encoder.fit(train_test_df[column])
    train_df[column] = column_encoder.transform(train_df[column])
    test_df[column] = column_encoder.transform(test_df[column])

In [42]:
tr_df, val_df = train_test_split(train_df, test_size=0.2, shuffle=False)

In [43]:
tr_song_df = tr_df.merge(songs_df, how='left', on='song_id')
tr_song_msno_df = tr_song_df.merge(members_df, how='left', on='msno')
val_song_df = val_df.merge(songs_df, how='left', on='song_id')
val_song_msno_df = val_song_df.merge(members_df, how='left', on='msno')
ts_song_df = test_df.merge(songs_df, how='left', on='song_id')
ts_song_msno_df = ts_song_df.merge(members_df, how='left', on='msno')

In [50]:
tr_song_msno_df.to_csv('../data/prep_tr_song_msno.csv',
                       encoding='utf-8',
                       index=False)
val_song_msno_df.to_csv('../data/prep_val_song_msno.csv',
                        encoding='utf-8',
                        index=False)
ts_song_msno_df.to_csv('../data/prep_ts_song_msno.csv',
                       encoding='utf-8',
                       index=False)

## Construct dataset

In [2]:
tr_song_msno_df = pd.read_csv("../data/prep_tr_song_msno.csv")
val_song_msno_df = pd.read_csv("../data/prep_val_song_msno.csv")
ts_song_msno_df = pd.read_csv("../data/prep_ts_song_msno.csv")

with open('../data/item2idx.json', 'r', encoding='utf-8') as f:
    item2idx = json.load(f)

In [3]:
full_df = pd.concat([tr_song_msno_df, val_song_msno_df, ts_song_msno_df],
                    ignore_index=True)

In [8]:
a = tr_song_msno_df['genre_ids'].apply(ast.literal_eval)

In [9]:
a[0]

[12]

In [74]:
sparse_features = [
    'msno',
    'song_id',
    'source_system_tab',
    'source_screen_name',
    'source_type',
    'composer',
    'lyricist',
    'language',
    'is_featured',
    'city',
    'registered_via',
    'gender',
]
dense_features = [
    'song_length',
    'num_artist',
    'num_composer',
    'num_lyricist',
    'bd',
    'registration_init_time',
    'expiration_date',
    'duration',
]
varlen_features = [
    'genre_ids',
    'artist_name',
]
embed_dim = 64

In [75]:
fixlen_feature_columns = [
    SparseFeat(feat, len(full_df[feat].unique()), embedding_dim=embed_dim)
    for feat in sparse_features
] + [DenseFeat(feat, 1) for feat in dense_features]

In [76]:
varlen_feature_columns = [
    VarLenSparseFeat(SparseFeat(feat,
                                vocabulary_size=len(item2idx[feat]),
                                embedding_dim=embed_dim),
                     maxlen=max(full_df[feat].apply(len)),
                     combiner='mean') for feat in varlen_features
]

In [77]:
linear_feature_columns = fixlen_feature_columns \
            + varlen_feature_columns
dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [78]:
tr_model_input = {name: tr_song_msno_df[name] for name in feature_names}
val_model_input = {name: val_song_msno_df[name] for name in feature_names}
ts_model_input = {name: ts_song_msno_df[name] for name in feature_names}

for feat in varlen_features:
    tr_model_input[feat] = pad_sequences(tr_song_msno_df[feat],
                                         maxlen=max(full_df[feat].apply(len)),
                                         padding='post')
    val_model_input[feat] = pad_sequences(val_song_msno_df[feat],
                                          maxlen=max(full_df[feat].apply(len)),
                                          padding='post')
    ts_model_input[feat] = pad_sequences(ts_song_msno_df[feat],
                                         maxlen=max(full_df[feat].apply(len)),
                                         padding='post')

In [79]:
model = DeepFM(linear_feature_columns,
               dnn_feature_columns,
               task='binary',
               l2_reg_embedding=1e-5,
               dnn_dropout=0.3,
               l2_reg_dnn=1e-5,
               dnn_use_bn=True,
               device=DEVICE)

In [80]:
model.compile(
    optimizer="adam",
    # optimizer=optim.RMSprop(model.parameters(), lr=1e-3),
    loss="binary_crossentropy",
    metrics=['auc'])

In [ ]:
history = model.fit(tr_model_input,
                    tr_song_msno_df['target'].values,
                    batch_size=8192,
                    epochs=10,
                    verbose=2,
                    validation_data=(val_model_input,
                                     val_song_msno_df['target'].values),
                    shuffle=True)